In [1]:
!pip install PyDrive
!pip install rouge

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import torch
import torch.nn as nn
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.__version__)

cuda:0
1.1.0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# #Data for training set = 10k dev set = 1k
# download = drive.CreateFile({'id': '1yJB36Jj1jw3U0JwE3IwhBYDY3VI1oCGd'})
# download.GetContentFile('train.p')
# download = drive.CreateFile({'id': '1T6YxqByvjNlCSZrU7FVmdCYlxM0RHlgw'})
# download.GetContentFile('dev.p')
# download = drive.CreateFile({'id': '1Nja8rXISqKfTSEbIPKYaf6Ey_Ud_Fik7'})
# download.GetContentFile('test.p')
# download = drive.CreateFile({'id': '1vPOGkgumTOVQ4N84GBo8AAPeAgwfyMep'})
# download.GetContentFile('index2word.p')
# download = drive.CreateFile({'id': '1JRYv-RQqhX6-fdYVUmyZmP9tBOtZjp_w'})
# download.GetContentFile('word2index.p')

In [0]:
# #Data with Extractive/Abstractive lable
download = drive.CreateFile({'id': '1LOzhGMuW8F4ChPUsycBjEOwpLDQPeD0I'})
download.GetContentFile('trainNew.p')
download = drive.CreateFile({'id': '1yoaCyqZgZ5QszX0g4ABnNV-9p-D6DUlZ'})
download.GetContentFile('devNew.p')
download = drive.CreateFile({'id': '1XSaEgXAwPWTojc6ZjzIy8vR38jjiCUSW'})
download.GetContentFile('testNew.p')
download = drive.CreateFile({'id': '1U45R7EOqcJCUQAyHMMRmhFgWoxfzxyL8'})
download.GetContentFile('index2wordNew.p')
download = drive.CreateFile({'id': '1DGnwq99nJAq9i9TQ45HPjaO3aYWqDDEL'})
download.GetContentFile('word2indexNew.p')

In [0]:
train_dict = pickle.load( open( "trainNew.p", "rb" ) )
dev_dict = pickle.load( open( "devNew.p", "rb" ) )
test_dict = pickle.load( open( "testNew.p", "rb" ) )
index2word = pickle.load( open( "index2wordNew.p", "rb" ) )
word2index = pickle.load( open( "word2indexNew.p", "rb" ) )

In [7]:
# data_code = "extractive"
data_code = "abstractive"
# data_code = "mixed"
filtered_train_dict = []
filtered_dev_dict = []
filtered_test_dict = []

for ind in range(len(train_dict)):
    data = train_dict[ind]
    if data['density_bin'] == data_code:
        filtered_train_dict.append(data)
for ind in range(len(dev_dict)):
    data = dev_dict[ind]
    if data['density_bin'] == data_code:
        filtered_dev_dict.append(data)
for ind in range(len(test_dict)):
    data = test_dict[ind]
    if data['density_bin'] == data_code:
        filtered_test_dict.append(data)

print(len(filtered_train_dict))
print(len(filtered_dev_dict))
print(len(filtered_test_dict))

5903
574
609


In [0]:
train_dict = filtered_train_dict[:]
dev_dict = filtered_dev_dict[:]
test_dict = filtered_test_dict[:]

In [0]:
# train_dict = pickle.load( open( "train.p", "rb" ) )
# dev_dict = pickle.load( open( "dev.p", "rb" ) )
# test_dict = pickle.load( open( "test.p", "rb" ) )
# index2word = pickle.load( open( "index2word.p", "rb" ) )
# word2index = pickle.load( open( "word2index.p", "rb" ) )

In [0]:
train_dict = sorted(train_dict, key = lambda i: len(i['text']), reverse = True)
dev_dict = sorted(dev_dict, key = lambda i: len(i['text']), reverse = True)
test_dict = sorted(test_dict, key = lambda i: len(i['text']), reverse = True)

In [0]:
train_dict = train_dict[:]
dev_dict = dev_dict[:]
test_dict = test_dict[:]

In [12]:
train_dict_text = []
for i, data in enumerate(train_dict):
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
#         if i==789:
#             print(s, summ)
    t['summary'] = summ
    train_dict_text.append(t)
print(len(train_dict_text))

dev_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    dev_dict_text.append(t)
print(len(dev_dict_text))

test_dict_text = []
for data in test_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    test_dict_text.append(t)
print(len(test_dict_text))

5903
574
609


In [13]:
ind = 0
print(train_dict[ind]['summary'])
print(train_dict_text[ind]['summary'])
print(index2word[train_dict[ind]['summary'][-2]])
print(test_dict[ind]['summary'])
print(test_dict_text[ind]['summary'])
print(index2word[test_dict[ind]['summary'][-2]])

[9938, 1003, 1215, 370, 38, 75273, 46, 9420]
inspiration can come from the unlikeliest of places
of
[13, 4663, 239, 6050, 3003, 21, 489, 46, 2728, 3229, 52, 12668, 1146, 21, 37, 21, 2063, 11, 50]
a blackout that cut power to all of south australia has prompted thousands to take to social media .
media


In [14]:
import nltk
import nltk.data
from nltk import tokenize
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pickle
import json, gzip
import time
import math
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import euclidean
nltk.download('punkt')

W0509 21:49:48.916234 140140726740864 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [16]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Instructions for updating:
Colocations handled automatically by placer.


W0509 21:50:07.468341 140140726740864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module(module_url)
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

In [18]:
ext_train_data = []
ext_dev_data = []
ext_test_data = []

for i, data in enumerate(train_dict_text):
    if i%1000==0:
        print(i)
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    try:
        sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
        num_sent = sentences_embeddings.shape[0]
        k = round(math.sqrt(num_sent))
        clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
        cluster_centers = clustering.cluster_centers_
        center_dist = defaultdict(list)
        for ns in range(num_sent):
            label = clustering.labels_[ns]
            dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
            center_dist[label].append((dist, ns))

        final = []
        for labels in range(k):
            try:
                center_dist[labels].sort(key = lambda x: x[0])
                final.append(center_dist[labels][0][1])
            except:
                continue
        final.sort()
        ex = []
        for i in final:
            ex.append(sentences_list[i])
        extractive = (' ').join(ex)
        dict_ = {}
        dict_['text'] = extractive[:]
        dict_['summary'] = data['summary'][:]
        ext_train_data.append(dict_)
    except:
        print('error')
end = time.time()
print(end-start)

for i, data in enumerate(dev_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_dev_data.append(dict_)
end = time.time()
print(end-start)

for i, data in enumerate(test_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_test_data.append(dict_)
end = time.time()
print(end-start)

0
1000
2000
3000
4000
5000
error
0.012334108352661133
0.01705145835876465
0.018419742584228516


In [0]:
ext_train_data = sorted(ext_train_data, key = lambda i: len(i['text']), reverse = True)
ext_dev_data = sorted(ext_dev_data, key = lambda i: len(i['text']), reverse = True)
ext_test_data = sorted(ext_test_data, key = lambda i: len(i['text']), reverse = True)

In [0]:
ind = 2
print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])
print(ext_dev_data[ind]['text'])
print(ext_dev_data[ind]['summary'])
print(ext_test_data[ind]['text'])
print(ext_test_data[ind]['summary'])

In [0]:
from nltk.tokenize import word_tokenize
ind2word = {0: 'PAD', 1:'BOS', 2:'EOS', 3:'UNK'}
word2ind = {'PAD':0, 'BOS':1, 'EOS':2, 'UNK':3}
count = 4
for data in ext_train_data:
    text = data['text'][:]
    summ = data['summary'][:]
    for words in word_tokenize(text):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

    for words in word_tokenize(summ):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

In [0]:
ind = 2
# print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])

In [0]:
train_dict = []
dev_dict = []
test_dict = []

for i, data in enumerate(ext_train_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(text):
        text_ind.append(word2ind[words])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))
        
    for words in word_tokenize(summ):
        summ_ind.append(word2ind[words])
    summ_ind = [1]+summ_ind+[2]
    data_point['summ_len'] = len(summ_ind)
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
        
    if len(summ_ind)>30:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        train_dict.append(data_point)
    
for data in ext_dev_data:
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    for words in word_tokenize(text):
        if words in word2ind:
            text_ind.append(word2ind[words])
        else:
            text_ind.append(word2ind['UNK'])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))
    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    data_point['summ_len'] = len(summ_ind)
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
    if len(summ_ind)>30:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        dev_dict.append(data_point)
        
for data in ext_test_data:
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    for words in word_tokenize(text):
        if words in word2ind:
            text_ind.append(word2ind[words])
        else:
            text_ind.append(word2ind['UNK'])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))
    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    data_point['summ_len'] = len(summ_ind)
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
    if len(summ_ind)>30:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        test_dict.append(data_point)

In [0]:
data = {}
data['train'] = train_dict[:]
data['dev'] = dev_dict[:]
data['test'] = test_dict[:]
s = (data['test'][9]['summary'])[:]
print(s)
st = ''
for ss in s:
    st += ind2word[ss]
    st += ' '
print(st)
print(ind2word[394])
#print(ind2word[184],ind2word[6398],ind2word[125],ind2word[93],ind2word[249],ind2word[18],ind2word[4500],ind2word[14327],ind2word[2586],ind2word[1815],ind2word[68],ind2word[660],ind2word[14] )

In [0]:
# #Padding Train and Dev Set
# data = {}
# train_data = []
# for obj in train_dict:
#     t = {}
#     a = obj['text'].copy()
#     b = obj['summary'].copy()
#     t['text_len'] = len(obj['text'])
#     while(len(a)<200):    
#         a.append(0)
#     t['text'] = a.copy()
#     b = [1] + b + [2] 
#     t['summ_len'] = len(b)
#     while(len(b)<22):    
#         b.append(0)
#     t['summary'] = b.copy()
#     train_data.append(t)
# data['train'] =  train_data[:]   


# dev_data = []
# for obj in dev_dict:
#     t = {}
#     a = obj['text'].copy()
#     b = obj['summary'].copy()
#     t['text_len'] = len(obj['text'])
#     while(len(a)<200):    
#         a.append(0)
#     t['text'] = a.copy()
#     b = [1] + b + [2] 
#     t['summ_len'] = len(b)
#     while(len(b)<22):    
#         b.append(0)
#     t['summary'] = b.copy()
#     dev_data.append(t)
# data['dev'] =  dev_data[:]      

# test_data = []
# for obj in test_dict:
#     t = {}
#     a = obj['text'].copy()
#     b = obj['summary'].copy()
#     t['text_len'] = len(obj['text'])
#     while(len(a)<200):    
#         a.append(0)
#     t['text'] = a.copy()
#     b = [1] + b + [2] 
#     t['summ_len'] = len(b)
#     while(len(b)<22):    
#         b.append(0)
#     t['summary'] = b.copy()
#     test_data.append(t)
# data['test'] =  test_data[:]      

In [0]:
trunc_norm_init_std = 1e-4
def init_linear_wt(linear):
    linear.weight.data.normal_(std=trunc_norm_init_std)
    if linear.bias is not None:
        linear.bias.data.normal_(std=trunc_norm_init_std)

In [0]:
class biEncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(biEncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers, bidirectional = True)
        self.reduce_h = nn.Linear(self.hidden_size * 2, self.hidden_size)
        init_linear_wt(self.reduce_h)
        self.reduce_c = nn.Linear(self.hidden_size * 2, self.hidden_size)
        init_linear_wt(self.reduce_c)


    def forward(self, input, hidden, memory, seq_lengths):
        batch, seq_len = input.size()
        emb = self.embedding(input)
        packed_emb = pack_padded_sequence(emb, (seq_lengths).cpu().numpy(), batch_first = True, enforce_sorted=False)
        packed_output, (h, c) = self.lstm(packed_emb, (hidden, memory))
        output, _ = pad_packed_sequence(packed_output, batch_first = True)
        output1 = output[ :, :, :self.hidden_size]
        output2 = output[:, :, self.hidden_size:] #reverse
        output = output1+output2
        h = torch.cat(list(h), dim=1)                                   
        c = torch.cat(list(c), dim=1)
        h_reduced = F.relu(self.reduce_h(h))                       
        c_reduced = F.relu(self.reduce_c(c))
        return output, h_reduced.unsqueeze(dim=0), c_reduced.unsqueeze(dim=0)

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.batch_size = params['batch_size']
        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first=True, num_layers = self.num_layers)
        self.w = nn.Linear(2*self.hidden_size, self.vocab_size)
        self.p_gen_linear = nn.Linear(self.hidden_size + self.emb_size, 1)

    def forward(self, input, hidden, memory, encoder_outputs, pad_ind):
        pointer_gen = True
        batch_size, seq_len = input.size()
        embedded = self.embedding(input)
        output,(h,c) = self.lstm(embedded, (hidden, memory))
        attn_values = torch.bmm(output,encoder_outputs.permute(0,2,1))
        dec_attn_values = torch.bmm(output,output.permute(0,2,1))
        for b in range(batch_size):
            attn_values[b,:,pad_ind[b]:] = -1e10
        mask = torch.ones_like(dec_attn_values).cuda()
        mask1 = -1e10*torch.ones_like(dec_attn_values).cuda()
        
        mask = torch.tril(mask, diagonal = -1)
        mask1 = torch.triu(mask1)
        mask = mask+mask1
        dec_attn_values = dec_attn_values*mask.float()
        
        total_attn_values = torch.cat((attn_values,dec_attn_values),2)
        total_outputs = torch.cat((encoder_outputs,output),1)
        
        sftmax = nn.Softmax(dim=2)
        attn_dist = sftmax(attn_values)# This is attention distribution
        
        total_attn_weights = sftmax(total_attn_values) 
        
        weight_attn = torch.bmm(total_attn_weights, total_outputs)
        final_hidden = torch.cat((output,weight_attn),2)
        scores = self.w(final_hidden)
        sftmax2 = nn.Softmax(dim=1)
        vocab_dist = sftmax2(scores)
        
        p_gen = None
        if pointer_gen:
            p_gen_input = torch.cat((embedded, output), 2)  # B x (hidden_dim + emb_dim)
            p_gen = self.p_gen_linear(p_gen_input)
            p_gen = torch.sigmoid(p_gen)
        vocab_dist_ = p_gen * vocab_dist
        attn_dist_ = (1 - p_gen) * attn_dist  
        return vocab_dist_, attn_dist_


    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class SuperAwesome(nn.Module):
    def __init__(self, params):
        super(SuperAwesome, self).__init__()
        self.bienc = biEncoderLSTM(params).cuda()
        self.attn_dc = AttnDecoderLSTM(params).cuda()
        self.vocab_size = params['vocab_size']
        self.bienc.embedding.weight = self.attn_dc.embedding.weight
    def forward(self, batch_text, batch_summary, batch_pad_len):
        batch_size, seq_size = batch_text.size()
        h_init,c_init = self.bienc.initHidden(batch_size)
        enc_out, h_enc, c_enc = self.bienc(batch_text,h_init,c_init, batch_pad_len)
        h_init, c_init = h_enc.contiguous(),c_enc.contiguous()
        vocab_dist_, attn_dist_ = self.attn_dc(batch_summary,h_init,c_init,enc_out, batch_pad_len)
        return vocab_dist_, attn_dist_

In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
def train_lm(dataset, params, net, fscore = False):
    
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = len(dataset)    
    batches = [(start, start + params['batch_size']) for start in range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        ref_summ = []
        gen_summ = []
        count = 0
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            batch_text = np.zeros([len(batch), len(batch[0]['text'])])
            batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
            batch_pad_len = np.zeros(len(batch))
            batch_summ_len = np.zeros(len(batch))
            i = 0
            for d in batch:
                batch_text[i] = d['text'].copy()
                batch_summary[i] = d['summary'].copy()
                batch_pad_len[i] = d['text_len']
                batch_summ_len[i] = d['summ_len']
                i += 1
                
            batch_text = torch.from_numpy(batch_text).long().cuda()
            batch_summary = torch.from_numpy(batch_summary).long().cuda()
            batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
            batch_summ_len = torch.from_numpy(batch_summ_len).long().cuda()
            vocab_dist_, attn_dist_ = net(batch_text, batch_summary, batch_pad_len)
            _,_,l = attn_dist_.size()
            enc_batch_extend_vocab = batch_text.unsqueeze(1)
            enc_batch_extend_vocab = enc_batch_extend_vocab[:,:,:l]
            
            enc_batch_extend_vocab = enc_batch_extend_vocab.expand_as(attn_dist_)
            final_dist = vocab_dist_.scatter_add(2, enc_batch_extend_vocab, attn_dist_)
            
            targets = batch_summary[:, 1:].contiguous()
            final_dist = final_dist[:, :-1, :]
            gold_probs = torch.gather(final_dist, 2, targets.unsqueeze(2)).squeeze()
            nll = -torch.log(gold_probs +  1e-12)
            #remove padding loss
            for b in range(len(batch)):
                nll[b,batch_summ_len[b]-1:] = 0
            loss = torch.sum(nll)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            
            if fscore:
                logits = final_dist
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summary[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summary[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='

                    gen_summ.append(summ)
                    ref_summ.append(gold)
                    
        idx = 5
        logits = final_dist[idx,:,:].unsqueeze(0)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(batch_summary[idx,1:].size()[0]):
            gold_summ_last += ind2word[batch_summary[idx,l].item()] + ' '
        for l in range(words.size()[0]):
            summ_last += ind2word[words[l].item()] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        
        loss, perplex =  compute_loss(data['dev'], net, params['batch_size'], fscore = True)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev loss: %0.2f, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, loss, perplex))
        print('####################################################')

In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def compute_loss(dataset, net, bsz=10, fscore = False):
    
    num_examples = len(dataset)  
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    gen_summ = []
    ref_summ = []
    ep_loss = 0.
    count = 0
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        batch_pad_len = np.zeros(len(batch))
        batch_summ_len = np.zeros(len(batch))
        i = 0
        for d in batch:
                batch_text[i] = d['text']
                batch_summary[i] = d['summary']
                batch_pad_len[i] = d['text_len']
                batch_summ_len[i] = d['summ_len']
                i += 1
        batch_text = torch.from_numpy(batch_text).long().cuda()
        batch_summary = torch.from_numpy(batch_summary).long().cuda()
        batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        batch_summ_len = torch.from_numpy(batch_summ_len).long().cuda()
        
        vocab_dist_, attn_dist_ = net(batch_text, batch_summary, batch_pad_len)
        _,_,l = attn_dist_.size()
        
        enc_batch_extend_vocab = batch_text.unsqueeze(1)
        enc_batch_extend_vocab = enc_batch_extend_vocab[:,:,:l]
        enc_batch_extend_vocab = enc_batch_extend_vocab.expand_as(attn_dist_)
        final_dist = vocab_dist_.scatter_add(2, enc_batch_extend_vocab, attn_dist_)
            
        targets = batch_summary[:, 1:].contiguous()
        final_dist = final_dist[:, :-1, :]
        gold_probs = torch.gather(final_dist, 2, targets.unsqueeze(2)).squeeze()
        nll = -torch.log(gold_probs +  1e-12)
        #remove padding loss
        for b in range(len(batch)):
            nll[b,batch_summ_len[b]-1:] = 0
        loss = torch.sum(nll)
        ep_loss += loss.detach()
        ut = torch.nonzero(batch_summary).size(0)
        total_unmasked_tokens += ut
        
        if fscore:
                logits = final_dist
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summary[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summary[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='

                    gen_summ.append(summ)
                    ref_summ.append(gold)
        
    idx = 2  
    logits = final_dist[idx,:,:].unsqueeze(0)
    words = torch.argmax(logits, dim=2).squeeze(0)
    gold_summ_last = ''
    summ_last = ''
    for l in range(batch_summary[idx,1:].size()[0]):
        gold_summ_last += ind2word[batch_summary[idx,l].item()] + ' '
    for l in range(words.size()[0]):
        summ_last += ind2word[words[l].item()] + ' '        
    
    print('#############Dev###########')
    if fscore:
        rouge_score = calculate_rouge(gen_summ, ref_summ)
        print('rouge', rouge_score)
    print(gold_summ_last)
    print(summ_last)
    
    perplexity = torch.exp(ep_loss / total_unmasked_tokens).cpu()
    return ep_loss, perplexity.data


In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def test_lm(dataset, net, bsz=1):
    num_examples = len(dataset)  
    batches = [(start, start + bsz) for start in\
               range(0, num_examples, bsz)]
    nll = 0
    max_len = 22
    for b_idx, (start, end) in enumerate(batches):
        batch = dataset[start:end]
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        batch_pad_len = np.zeros(len(batch))
        batch_summ_len = np.zeros(len(batch))
        i = 0
        for d in batch:
                batch_text[i] = d['text']
                batch_summary[i] = d['summary']
                batch_pad_len[i] = d['text_len']
                batch_summ_len[i] = d['summ_len']
                i += 1
        batch_text = torch.from_numpy(batch_text).long().cuda()
        batch_summary = torch.from_numpy(batch_summary).long().cuda()
        batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        batch_summ_len = torch.from_numpy(batch_summ_len).long().cuda()
        batch_summary_test = torch.ones([bsz, 1], dtype=torch.long).cuda()
        ans = ''
        count = 0
        for i in range(max_len):    
            vocab_dist_, attn_dist_ = net(batch_text, batch_summary_test, batch_pad_len)
            _,_,l = attn_dist_.size()
        
            enc_batch_extend_vocab = batch_text.unsqueeze(1)
            enc_batch_extend_vocab = enc_batch_extend_vocab[:,:,:l]
            enc_batch_extend_vocab = enc_batch_extend_vocab.expand_as(attn_dist_)
            final_dist = vocab_dist_.scatter_add(2, enc_batch_extend_vocab, attn_dist_)
#             print(final_dist.size())
            final_dist = final_dist.squeeze(0)
            tx = torch.argmax(final_dist,1)
            temp = tx[-1]
            temp = temp.cpu()
            x = torch.ones([bsz, 1], dtype=torch.long).cuda()
            x[0][0] = temp.item()
            if(ind2word[temp.item()] == "EOS"):
                break
            ans += ind2word[temp.item()] + ' '
            count += 1
            if(ind2word[temp.item()] == "EOS"):
                break
            batch_summary_test = torch.cat((batch_summary_test,x),1) 
        #print("Insise test_lm: ")
        #print(ans)
    return ans, count

In [0]:
params = {}
params['hidden_size'] = 512
params['emb_size'] = 512
params['num_layers'] = 1
params['batch_size'] = 150
params['vocab_size'] = len(word2ind)
params['learning_rate'] = 0.001
params['epochs'] = 25
net = SuperAwesome(params)

train_lm(data['train'], params, net, fscore = True)

In [0]:
test_len = []
#TEST TIME
gen_summ = []
ref_summ = []
for i in range(len(data['test'])):
    #print(i, end=',')
    y = data['test'][i]['summary']
    gold = ''
    for t in y:
        if(ind2word[t]=='SOS'):
            continue
        if(ind2word[t] == 'EOS'):
            break  
        gold += (ind2word[t])
        gold +=' '
    
    
    print("ground truth")  
    print(gold)
    net.eval()
    summ, l = test_lm(data['test'][i:i+1], net) 
    test_len.append(l)
    if summ == '.'*len(summ):
        count += 1
    if not summ or summ == '.'*len(summ):
        summ = '====='
    print(summ)
    gen_summ.append(summ)
    ref_summ.append(gold)
rouge_score = calculate_rouge(gen_summ, ref_summ)
print('Test rouge', rouge_score)  
# for n in range(num_examples):
#         summ = ''
#         for i in range(1,max_len+1):
#             if summ:
#                 summ += ' '
#             if int(ans[n,i]) == 2:
#                 break
#             summ += ind2word[int(ans[n,i])]
#         gen_summ.append(summ)